In [ ]:
source("../shared.R")

In [ ]:
library("COUNT")
data("badhealth")

In [ ]:
head(badhealth)

In [ ]:
hist(badhealth$numvisit, breaks=20)

In [ ]:
plot(jitter(log(numvisit)) ~ jitter(age), data=badhealth, subset=badh==0, xlab="age", ylab="log(visits)")
points(jitter(log(numvisit)) ~ jitter(age), data=badhealth, subset=badh==1, col="red")

In [ ]:
str_mod = " model {
    for (i in 1: length(numvisit)) {
        numvisit[i] ~ dpois(lam[i])
        log(lam[i]) = int + b_badh*badh[i] + b_age*age[i] + b_intx*age[i]*badh[i]
    }    
    int ~ dnorm(0, 1e-6)
    b_badh ~ dnorm(0, 1e-6)
    b_age ~ dnorm(0, 1e-6)
    b_intx ~ dnorm(0, 1e-6)
} "

mod = jags_model_compile(str_mod, as.list(badhealth), burnout = 1e3 )

In [ ]:
params = c("int", "b_badh", "b_age", "b_intx")
modsamples  = jags_model_sample(mod, var_names_ = params, n_iter_ = 5e3)
moddiag = jags_model_diag(modsamples$samples)

In [ ]:
dic1 = dic.samples(mod, n.iter = 5e3)

In [ ]:
dic1

In [ ]:
str_mod2 = " model {
    for (i in 1: length(numvisit)) {
        numvisit[i] ~ dpois(lam[i])
        log(lam[i]) = int + b_badh*badh[i] + b_age*age[i]
    }    
    int ~ dnorm(0, 1e-6)
    b_badh ~ dnorm(0, 1e-6)
    b_age ~ dnorm(0, 1e-6)
    b_intx ~ dnorm(0, 1e-6)
} "
data2 = list(numvisit = badhealth$numvisit, badh = badhealth$badh, age = badhealth$age)
mod2 = jags_model_compile(str_mod2, data2, burnout = 1e3 )

In [ ]:
dic2 = dic.samples(mod2, n.iter = 5e3)

In [ ]:
dic2

In [ ]:
plot(density(modsamples$samples_cmb[,"b_badh"]))

## Model Checking

In [ ]:
X = as.matrix(badhealth[,-1])
X = cbind(X, with(badhealth, badh*age))
head(X)

In [ ]:
pmed_cff = apply(modsamples$samples_cmb, 2, median)
pmed_cff

In [ ]:
lam_hat = exp(pmed_cff['int'] + X %*% pmed_cff[c('b_badh', 'b_age', 'b_intx')])

In [ ]:
resid = badhealth$numvisit - lam_hat
plot(resid)

In [ ]:
plot(lam_hat, badhealth$numvisit)
abline(0.0, 1.0)

In [ ]:
good_healt = which(badhealth$badh == 0)
bad_healt = which(badhealth$badh == 1)
plot(lam_hat[good_healt], resid[good_healt], xlim=c(0,8))
points(lam_hat[bad_healt], resid[bad_healt], col="red")

In [ ]:
var(resid[good_healt]) # we expect the variance to be equal to poisson distr

In [ ]:
var(resid[bad_healt])

In [ ]:
summary(modsamples$samples_cmb)

In [ ]:
x1 = c(0, 35, 0)  # good health
x2 = c(1, 35, 35) # bad health

In [ ]:
sim = modsamples$samples_cmb
n_sim = length(sim)
lam1 = exp( sim[, "int"] + sim[, c(2,1,3)] %*% x1)
lam2 = exp( sim[, "int"] + sim[, c(2,1,3)] %*% x2)

In [ ]:
?rpois

In [ ]:
y1 = rpois(n=n_sim, lambda = lam1)
y2 = rpois(n=n_sim, lambda = lam2)

In [ ]:
plot(table(factor(y1, levels=0:18))/n_sim, pch=2, ylab="posterior prob.", xlab="visits")
points(table(y2+0.1)/n_sim, col="red")

In [ ]:
mean(y2 > y1)

In [ ]:
exp(1.5 - 0.3*0.8 + 1.2)

In [ ]:
y = rpois(n = 1e6,lambda = 30) 

In [ ]:
mean(y < 22)